# Cluster Validity

#### Garrett McCue

Goal of this notebook is to: 
1. Implement three cluster validity indices (one must be the CS index).
2. Taking into consideration the results you obtained after implementing VAT, iVAT (as a pre-clustering technique) and FCM (as a clustering technique), compare the results you will get after applying the validity indices with the previous results of VAT (iVAT) and FCM. What do you think the best choice for the number of clusters (C) will be?



The application of clustering algorithms to a dataset that does not have an optimal number of gropupings a prior can lead to the discovery of various amounts of underlying clusters. The issue with trying different numbers of clusters on the same dataset is that it can be difficult to discern what clustering is the best choice. The variation within cluster membership can be influenced by applying different clustering methods, or distance measures. The different clustering methods will still result with various clustering combinations, but still are without evidence of the best choice. In order to assess which clustering is the optimal choice, clustering validity indices can be computed by evaluating the "goodness" of the cluster set. The criterion for clustering goodness focuses on how well seprated each group is from one another and how compact or dense each grouping is. For a set of potential clustering numbers, or candidate parition set, the membership matrix, $\cup$, for each candidate partion is evaluated using a validity index. Depending on the clustering index that is being used, the optimal choice will yield the min/max index value when compared to with all candidates. Classification entropy and Partition coefficient are two popular validity indices that are relatively simple to implement, using only the membership matrices for each candidate partition. The relatively simplistic nature of these algorithms comes with a trade off becasue the lack of direct connection with each data point of the clusterings themselves,  results in evaluation scores that do not assess the data in its entirety. Other validity indices such as CS index provide a more robust evaluation metric by considering the distance within each cluster and the distance between each cluster. The CS index algorithm is more complex to implement than both, Classification entropy and Partition coefficient algorithms, which is trade off when considering more complex criteria for clustering validity. The implementation and results of both simple algorithms as well as CS index will be evaluated on the [2022 World Happiness Dataset](https://www.kaggle.com/datasets/hemil26/world-happiness-report-2022) after [Clustering with FCM](https://nbviewer.org/github/mcqueg/Unsupervised_ML/blob/main/FCM.ipynb) and will be compared to the [VAT/iVAT](https://nbviewer.org/github/mcqueg/Unsupervised_ML/blob/main/VAT.ipynb) pre-clustering analysis.

In [1]:
import numpy as np
import pandas as pd
from fcmeans import FCM
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
# import data
hap_df = pd.read_csv("data/world_happiness_rankings_2022.csv")
ranking_df = hap_df[['RANK', 'Country']]
metrics_df = hap_df.drop(['RANK', 'Country'], axis=1)

hap_df.head()

,RANK,Country,Happiness score,Whisker-high,Whisker-low,Dystopia (1.83) + residual,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption
0,1,Finland,7.821,7.886,7.756,2.518,1.892,1.258,0.775,0.736,0.109,0.534
1,2,Denmark,7.636,7.710,7.563,2.226,1.953,1.243,0.777,0.719,0.188,0.532
2,3,Iceland,7.557,7.651,7.464,2.320,1.936,1.320,0.803,0.718,0.270,0.191
3,4,Switzerland,7.512,7.586,7.437,2.153,2.026,1.226,0.822,0.677,0.147,0.461
4,5,Netherlands,7.415,7.471,7.359,2.137,1.945,1.206,0.787,0.651,0.271,0.419


In [3]:
# scale data
metrics_df = StandardScaler().fit_transform(metrics_df)

# apply 2D PCA to data
pca_2 = PCA(n_components=2)
pca_2_data = pca_2.fit_transform(metrics_df)
pca_2_df = pd.DataFrame(data=pca_2_data, columns=['PC1', 'PC2'])
pca_2_ranking_df = pd.concat([ranking_df, pca_2_df], axis=1)

# apply 3D PCA to data
pca_3 = PCA(n_components=3)
pca_3_data = pca_3.fit_transform(metrics_df)
pca_3_df = pd.DataFrame(data=pca_3_data, columns=['PC1', 'PC2', 'PC3'])
pca_3_ranking_df = pd.concat([ranking_df, pca_3_df], axis=1)

X_2d = pca_2_df.to_numpy()
X_3d = pca_3_df.to_numpy()

### Classification entropy
$N$ : number of data points  
$\mu_{ij}$ : the membership of the $i^\text{th}$ data point to the $j^\text{th}$  
$C$ : number of cluster centers
$$ CE(c) = -\frac{1}{N}\sum_{i=1}^{C}\sum_{j=1}^{N}log(\mu_{ij})\mu_{ij}$$


In [4]:
def ce(u):
    n, c = u.shape
    return abs((np.log10(u) * u).sum() / n)

In [5]:
print("2D Clustering: ")
for n_clusters in range(2, 6):
    fcm = FCM(n_clusters=n_clusters, m=2)
    fcm.fit(X_2d)
    u = fcm.u
    index = ce(u)

    print(f"classification entropy with {n_clusters} number of clusters is: {index}")

print("\n3D Clustering: ")
for n_clusters in range(2, 6):
    fcm = FCM(n_clusters=n_clusters, m=2)
    fcm.fit(X_3d)
    u = fcm.u
    index = ce(u)

    print(f"classification entropy with {n_clusters} number of clusters is: {index}")

2D Clustering: 
classification entropy with 2 number of clusters is: 0.15636960306534034
classification entropy with 3 number of clusters is: 0.26217202263157513
classification entropy with 4 number of clusters is: 0.31092082979104
classification entropy with 5 number of clusters is: 0.36385630813015407

3D Clustering: 
classification entropy with 2 number of clusters is: 0.18630143251697812
classification entropy with 3 number of clusters is: 0.29975887175241017
classification entropy with 4 number of clusters is: 0.3707407382212827
classification entropy with 5 number of clusters is: 0.4326485491159845


### Partition Coefficient

$N$ : number of data points  
$\mu_{ij}$ : the membership of the $i^\text{th}$ data point to the $j^\text{th}$  
$C$ : number of cluster centers
$$ PC(c) = \frac{1}{N}\sum_{i=1}^{C}\sum_{j=1}^{N}\mu_{ij}^2$$


In [6]:
def pc(u):
    n, c = u.shape
    return np.square(u).sum() / n

In [7]:
print("2D Clustering: ")
for n_clusters in range(2, 6):
    fcm = FCM(n_clusters=n_clusters, m=2)
    fcm.fit(X_2d)
    u = fcm.u
    index = pc(u)

    print(f"partition coeficient with {n_clusters} number of clusters is: {index}")

print("\n3D Clustering: ")
for n_clusters in range(2, 6):
    fcm = FCM(n_clusters=n_clusters, m=2)
    fcm.fit(X_3d)
    u = fcm.u
    index = pc(u)

    print(f"partition coeficient with {n_clusters} number of clusters is: {index}")

2D Clustering: 
partition coeficient with 2 number of clusters is: 0.7781284291005209
partition coeficient with 3 number of clusters is: 0.6568531487683938
partition coeficient with 4 number of clusters is: 0.6264371762867962
partition coeficient with 5 number of clusters is: 0.5800084653103927

3D Clustering: 
partition coeficient with 2 number of clusters is: 0.7298308226247467
partition coeficient with 3 number of clusters is: 0.6048014692255097
partition coeficient with 4 number of clusters is: 0.5508124501907692
partition coeficient with 5 number of clusters is: 0.5014878721381265


### CS Index

$C$ : number of cluster centers  
$A_i$:  the number of data points in the cluster i
$v_i$:  the cluster center of cluster i   
$d_{ij}$ : Euclidean distance between $i^\text{th}$ data point and $j^\text{th}$ data point


$$ CS(c) = \frac{\frac{1}{C}\sum_{i=1}^{C} \left\{ \frac{1}{|A_i|}\sum_{\vec{x_j}\epsilon{A_i}}\max_{\vec{x_j}\epsilon{A_i}}d(\vec{x_i},\vec{x_j}) \right\}}{\frac{1}{C}\sum_{i=1}^{C} \left\{ \min_{j\epsilon{C}_{j\neq{i}}} d(\vec{v_i},\vec{v_j}) \right\}} $$



In [8]:
def euclidean_dist(p,q):
    if len(p) == 2:
        distance = np.sqrt(((q[1]-p[1])**2)+((q[0]-p[0])**2))
    elif len(p) == 3:
        distance = np.sqrt(((q[1]-p[1])**2)+((q[0]-p[0])**2)+((q[2]-p[2])**2))
    return distance

In [9]:
def cs_index(X, centers, labels, n_clusters):
    # list to hold the number of observations in each cluster
    a = []
    # initialize empty list to hold the max distance within each cluster
    inner_max_dist_c = []
    # loop through each cluster calculating the max within-cluster distance
    for c in range(0, n_clusters):
        # initialize list to hold the max within-distance for current cluster
        inner_max_dist = []
        # create temp array to hold only the observations for the current cluster
        labels_c = np.squeeze(np.array(np.where(labels == c)))
        # add the number of data points in cluster to list a
        a.append(len(labels_c))
        X_c = []
        for k in labels_c:
            X_c.append(X[k])
        # loop through each observation within cluster saving distance of every sample vs all other samples
        for row in range(len(X_c)):
            # initialize list to hold all distances corresponding to the current observation
            distances = []
            # loop through all rows finding distance to current row
            for j in range(len(X_c)):
                # add distances to distances list for current row
                distances.append(euclidean_dist(X_c[row], X_c[j]))
            # find max distance for each sample vs all other samples
            inner_max_dist.append(max(distances))
        # find max distance for each clister
        inner_max_dist_c.append(max(inner_max_dist))
    # sum all max distances between points of each cluster
    sum_inner_max_dist_c = sum(inner_max_dist_c)
    # sum of max distances between points in each cluster divided by the size of each cluster
    sum_a = 0
    for i in range(len(a)):
        sum_a = sum_a + ((1/a[i])*sum_inner_max_dist_c)
    # cs_numerator: within cluster distance measures
    cs_numerator = (1/n_clusters) * sum_a

    # intitalize the sum of the between cluster distances as 0
    sum_between_cluster_dist = 0
    # loop through every cluster centroid to get the min distance between clsuter centroids
    for c in range(len(centers)):
        # initialize list to hold the distances between current cluster and all other clusters
        centroid_dists = []
        # find distances for each cluster centroid with current centroid
        for j in range(len(centers)):
            # add distance to list for current cluster
            centroid_dists.append(euclidean_dist(centers[c], centers[j]))
        # remove the distance measure of 0 (current cluster with itself)
        centroid_dists.remove(0)
        # add the min distance to the sum of between cluster distances
        sum_between_cluster_dist = sum_between_cluster_dist + \
            (min(centroid_dists))
    # cs_denominator: (1/n_clusters)*sum of between cluster distances
    cs_denominator = (1/n_clusters)*sum_between_cluster_dist
    # cs_index = cs_numerator/cs_denominator
    index = cs_numerator/cs_denominator

    return index


In [10]:
print("2D Clustering: ")
for n_clusters in range(2, 6):
    fcm = FCM(n_clusters=n_clusters, m=2)
    fcm.fit(X_2d)
    centers_2d = fcm.centers
    labels_2d = fcm.predict(X_2d)
    index = cs_index(X_2d, centers=centers_2d, labels=labels_2d, n_clusters=n_clusters)

    print(f"cs index with {n_clusters} number of clusters is: {index}")

print("\n3D Clustering: ")
for n_clusters in range(2, 6):
    fcm = FCM(n_clusters=n_clusters, m=2)
    fcm.fit(X_3d)
    centers_3d = fcm.centers
    labels_3d = fcm.predict(X_3d)
    index = cs_index(X_3d, centers=centers_3d, labels=labels_3d, n_clusters=n_clusters)

    print(f"cs index with {n_clusters} number of clusters is: {index}")

2D Clustering: 
cs index with 2 number of clusters is: 0.04421837449759678
cs index with 3 number of clusters is: 0.12854420019334895
cs index with 4 number of clusters is: 0.2357542140867573
cs index with 5 number of clusters is: 0.38731534032540216

3D Clustering: 
cs index with 2 number of clusters is: 0.04905895861605767
cs index with 3 number of clusters is: 0.14109983855117003
cs index with 4 number of clusters is: 0.2652773642084752
cs index with 5 number of clusters is: 0.40310910586834714


## Cluster Validity Analysis

#### Classification Entropy
When applying the Classification Entropy to the 2D and 3D PCA projected data, the optimal number of clusters is given by the lowest index value for the number of clusters. The lowest classification entropy of the 2D data was 0.156, resulting in the optimal number of clusters being 2. The classifcation entropy of the 3D data also resulted in the optimal clsutering number of 2, with a minimum index value of 0.186. When comparing the two index values it seems that 2 clusters within the 2D dataset is a better clustering than 2 clusters within the 3D dataset. 

#### Partition Coefficient 
When applying the Partition Coefficient to the 2D and 3D PCA projected data, the optimal number of clusters is given by the largest index value for the number of clusters. The largest partition Coefficient of the 2D data was 0.778, resulting in the optimal number of clusters being 2. The Partition Coefficient of the 3D data also resulted in the optimal clsutering number of 2, with a max index value of 0.729. When comparing the two index values it seems that 2 clusters within the 2D dataset is a better clustering than 2 clusters within the 3D dataset. 

#### CS Index
When applying the CS Index to the 2D and 3D PCA projected data, the optimal number of clusters is given by the lowest index value for the number of clusters. The lowest CS Index of the 2D data was 0.044, resulting in the optimal number of clusters being 2. The CS Index of the 3D data also resulted in the optimal clsutering number of 2, with a minimum index value of 0.049. When comparing the two index values it seems that 2 clusters within the 2D dataset is a better clustering than 2 clusters within the 3D dataset. 

#### Comparison with VAT/iVAT Analysis
When the VAT/iVAT algorithms were applied to the 2D and 3D projected data the analysis resulted in images suggesting the presence of 3, 4, and potentially 5 clusters. The FCM clustering algorithm was applied to the 2D and 3D projected data searching for the optimal number of clusters based on the suggested clustering numbers generated by the preclustering analysis. When the cluster validity indices were computed for all possible clusterings from the FCM algorithm it was determined that a smaller number of clusters were optimal for this dataset. The indices were also computed for a clustering with 2 groups, which was not the suggested clustering number assessed from the VAT/iVAT algorithm. This can serve as a reminder that the pre-clustering analysis is only a guide for the datasets ability to cluster and is not a definitive guide for the optimal clustering number. 
